In [1]:
import numpy as np
import pandas as pd

train_data = pd.read_csv('income_train.csv')
test_data = pd.read_csv('income_test.csv')

In [9]:
train_data['Category'] = train_data['Category'].apply(lambda x : +1 if x=='>50K' else -1)
train_data = pd.get_dummies(train_data)

features = list(train_data.columns[train_data.columns != 'Id'])
features.remove('Category')
features

['age',
 'fnlwgt',
 'education-num',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'workclass_?',
 'workclass_Federal-gov',
 'workclass_Local-gov',
 'workclass_Never-worked',
 'workclass_Private',
 'workclass_Self-emp-inc',
 'workclass_Self-emp-not-inc',
 'workclass_State-gov',
 'workclass_Without-pay',
 'education_10th',
 'education_11th',
 'education_12th',
 'education_1st-4th',
 'education_5th-6th',
 'education_7th-8th',
 'education_9th',
 'education_Assoc-acdm',
 'education_Assoc-voc',
 'education_Bachelors',
 'education_Doctorate',
 'education_HS-grad',
 'education_Masters',
 'education_Preschool',
 'education_Prof-school',
 'education_Some-college',
 'marital-status_Divorced',
 'marital-status_Married-AF-spouse',
 'marital-status_Married-civ-spouse',
 'marital-status_Married-spouse-absent',
 'marital-status_Never-married',
 'marital-status_Separated',
 'marital-status_Widowed',
 'occupation_?',
 'occupation_Adm-clerical',
 'occupation_Armed-Forces',
 'occupation_Craft-rep

In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

n_estimators = [50,100,150,200,300]
learning_rate = [0.05,0.1,0.25,0.5,0.75,1]
hyperparameters = {'n_estimators': n_estimators,
                   'learning_rate': learning_rate}
search = GridSearchCV(estimator=GradientBoostingClassifier(n_estimators=...,
                                                   learning_rate=...),
                     param_grid=hyperparameters,
                     cv=6,
                     return_train_score=True)
best_params = search.fit(train_data[features], train_data['Category']).best_params_


In [20]:
gradient_model = GradientBoostingClassifier(n_estimators=best_params['n_estimators'],
                                    learning_rate=best_params['learning_rate'])
gradient_model.fit(train_data[features], train_data['Category'])

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=300,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [21]:
# Decision Tree Predict
test_data = pd.get_dummies(test_data)
gradient_prediction = gradient_model.predict(test_data[features])
predictions = []
for i in range(len(gradient_prediction)):
    if gradient_prediction[i] == 1:
        predictions.append('>50K')
    else: 
        predictions.append('<=50K')

to_save = test_data[['Id']].copy()
to_save.loc[:, 'Category'] = predictions
to_save.to_csv('submission_gradientboost.csv', index=False)

